In [73]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [74]:
class Pretrain(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv2d(1, 32, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.fc1 = nn.Linear(256, 120)
        self.fc2 = nn.Linear(120, 10)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        return x

In [75]:
class FF(nn.Module):
    def __init__(self, model):
        super().__init__()
        
        self.cnn = model
        
        self.fc = nn.Linear(20, 1)
        
    def forward(self, x):
        x1 = x[:,:1,:]
        x2 = x[:,1:2,:]
        

        x1 = self.cnn(x1)
        x2 = self.cnn(x2)
            
        x = self.fc(torch.cat((x1, x2), dim=1))
        
        return x

In [76]:
model = Pretrain()

In [77]:
from dataloading import load_data

In [78]:
trainloader, testloader = load_data()

In [79]:
learning_rate = .001

criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(baseline.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [80]:
epochs = 10

In [81]:
losses = []

for epoch in range(epochs):
    running_loss = 0.0
    epoch_losses = []
    
    for i, data in enumerate(trainloader, 0):
        inputs, target, classes = data
        
        x1 = inputs[:,:1,:]
        x2 = inputs[:,1:2,:]
        
        optimizer.zero_grad()
        
        outputs = model(x1)
        outputs = outputs
        
        loss = criterion(outputs, classes[:,0])
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        epoch_losses.append(loss.item())
    
    epoch_loss = torch.mean(torch.tensor(epoch_losses))
    print(f'epoch: {epoch}, loss: {epoch_loss}')
    losses.append(epoch_loss)  

epoch: 0, loss: 3.686640501022339
epoch: 1, loss: 0.5478036403656006
epoch: 2, loss: 0.2662648856639862
epoch: 3, loss: 0.1879831999540329
epoch: 4, loss: 0.11312808841466904
epoch: 5, loss: 0.06492424011230469
epoch: 6, loss: 0.0596708245575428
epoch: 7, loss: 0.03277947008609772
epoch: 8, loss: 0.016993051394820213
epoch: 9, loss: 0.009113973006606102


In [82]:
ff = FF(model)

In [83]:
learning_rate = .001

criterion = nn.BCEWithLogitsLoss()
# optimizer = optim.SGD(baseline.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(ff.parameters(), lr=learning_rate)

In [84]:
losses = []

for epoch in range(epochs):
    running_loss = 0.0
    epoch_losses = []
    
    for i, data in enumerate(trainloader, 0):
        inputs, target, classes = data
        
        optimizer.zero_grad()
        
        outputs = ff(inputs)
        outputs = outputs.squeeze()
        
        loss = criterion(outputs, target.float())
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        epoch_losses.append(loss.item())
    
    epoch_loss = torch.mean(torch.tensor(epoch_losses))
    print(f'epoch: {epoch}, loss: {epoch_loss}')
    losses.append(epoch_loss)  

epoch: 0, loss: 0.5458245277404785
epoch: 1, loss: 0.3479655683040619
epoch: 2, loss: 0.25985392928123474
epoch: 3, loss: 0.18957576155662537
epoch: 4, loss: 0.15485483407974243
epoch: 5, loss: 0.12116918712854385
epoch: 6, loss: 0.11432626843452454
epoch: 7, loss: 0.12972129881381989
epoch: 8, loss: 0.07614415138959885
epoch: 9, loss: 0.04117998480796814


In [85]:
from evaluate import evaluate_model

evaluate_model(ff, testloader)

Accuracy: 0.8619999885559082
